In [1]:
from omnia_ae.api import AEEnvironment, AEAPI

In [2]:
from azure.identity import ClientSecretCredential
import os
credential = ClientSecretCredential(
    tenant_id=os.environ['AZURE_TENANT_ID'],
    client_id=os.environ['AZURE_CLIENT_ID_NON_PRODUCTION'],
    client_secret=os.environ['AZURE_CLIENT_SECRET_NON_PRODUCTION']
)

In [3]:
api = AEAPI(azure_credential=credential, environment=AEEnvironment.Dev())

In [4]:
api.get_sources()

{'items': [{'facility': 'AHA'},
  {'facility': 'ASGA'},
  {'facility': 'ASGB'},
  {'facility': 'GFA'},
  {'facility': 'GFB'},
  {'facility': 'GFC'},
  {'facility': 'GKR'},
  {'facility': 'HD'},
  {'facility': 'JSV'},
  {'facility': 'KRIS'},
  {'facility': 'MARA'},
  {'facility': 'MARB'},
  {'facility': 'MLA'},
  {'facility': 'NOR'},
  {'facility': 'OSF'},
  {'facility': 'TROA'},
  {'facility': 'VIS'}]}

In [12]:
api.get_events("AHA", "*", 1, includeOtherFields=False)

{'items': [{'timestamp': '2016-02-23T05:06:55.4670000Z',
   'sourceName': '71UI2601',
   'description': 'DV lower deck south released',
   'eventType': 'Simple',
   'eventCategory': '3439269159',
   'message': 'Normal state',
   'severity': 302,
   'priority': None,
   'condition': None,
   'alarmState': None,
   'ackedState': None,
   'activeState': None,
   'suppressedOrShelved': None,
   'node': 'A',
   'processArea': 'FG',
   'actiontime': '2016-02-23T05:06:55.4670000Z'}],
 'continuationToken': 'eyJPZmZzZXQiOjF9'}

In [11]:
api.get_realtime_subscriptions()

{'data': {'items': [{'facility': 'AHA'}, {'facility': 'NOR'}]}}

In [7]:
api.set_realtime_subscription(connectionString="test"
)
#statusCode: 200, with real connection string

{'statusCode': 500,
 'message': 'Connection string is invalid',
 'traceId': '0HMV52IUBNO12:00000088'}

In [10]:
api.create_subscription(facility="NOR")

{'statusCode': 200, 'message': 'Created subscription NOR'}

In [9]:
api.delete_subscription(facility="NOR")

{'statusCode': 200, 'message': 'Deleted subscription NOR'}